$$ ITI \space AI: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 1 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [ ]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [ ]:
from itertools import combinations
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

----------------------------

# `02` Load Data

In [ ]:
ratings = pd.read_csv("songsDataset.csv", names=['userID', 'songID', 'rating'], skiprows=[0])
ratings.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


---------------------------------

# `03` Similarity Metrics

## `0` Utility Matrix
Construct utility matrix for the loaded data `ratings`
- Users as Index
- Songs as Columns

**Hint**: you can use `pandas.DataFram.pivot` method (see [Documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html))

In [ ]:
utility_matrix = ratings.pivot(index='userID', columns='songID', values='rating')

In [ ]:
utility_matrix["mean_user_rating"]=utility_matrix.mean(axis=1)

In [ ]:
utility_matrix.fillna(0,inplace=True)

In [ ]:
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,119103,120147,122065,123176,125557,126757,131048,132189,134732,mean_user_rating
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.333333
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.000000
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.000000
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.500000


## `i` Cosine Similarity
Finish implmenting the function below to calculate `Cosine Similarity` between two vectors

In [ ]:
def cosine_sim(vec_a, vec_b):
    """
    Returns the raw cosine similarity score between two vectors.

            Parameters:
                vec_a (pandas.Series): Vector A
                vec_b (pandas.Series): Vector B

            Returns:
                sim_score (float): Similarity score between vectors vec_a and vec_b
    """

    vec_a_cleaned=vec_a[:-1].copy()
    vec_b_cleaned=vec_b[:-1].copy()
    dot_product=vec_a_cleaned @ vec_b_cleaned
    a=np.sqrt(np.square(vec_a_cleaned).sum())
    b=np.sqrt(np.square(vec_b_cleaned).sum())
    sim_score = dot_product/(a*b)
    return sim_score

In [ ]:
print(f'Cosine Similarity between userID 56 and userID 227 is: {cosine_sim(utility_matrix.iloc[56].copy(), utility_matrix.iloc[227].copy())}')

Cosine Similarity between userID 56 and userID 227 is: 0.7808688094430304


## `ii` Adjusted Cosine Similarity
Finish implmenting the function below to calculate `Adjusted Cosine Similarity` between two vectors

In [ ]:
def adjusted_cosine_sim(vec_a, vec_b,mean_user_rating):
    """
    Returns the adjusted cosine similarity score between two vectors.

            Parameters:
                vec_a (pandas.Series): Vector A
                vec_b (pandas.Series): Vector B

            Returns:
                sim_score (float): Similarity score between vectors vec_a and vec_b
    """
    common_users = (vec_a!= 0.0) & (vec_b != 0.0)
    vec_a_common =vec_a[common_users]
    vec_b_common =vec_b[common_users]
    filtered_mean=mean_user_rating[common_users]
    adjusted_rating_a=vec_a_common-filtered_mean
    adjusted_rating_b=vec_b_common-filtered_mean
    dot_product = adjusted_rating_a @ adjusted_rating_b
    a=np.sqrt((np.square(adjusted_rating_a)).sum())
    b=np.sqrt((np.square(adjusted_rating_b)).sum())
    if a==0 or b==0:
        return 0
    sim_score = dot_product/(a*b)
    return sim_score

In [ ]:
#print(f'Adjusted Cosine Similarity between userID 56 and userID 227 is: {adjusted_cosine_sim(utility_matrix.iloc[56].copy(), utility_matrix.iloc[227].copy(), utility_matrix["mean_user_rating"].copy())}')
print(f'Adjusted Cosine Similarity between songID 3785 and songID 17029 is: {adjusted_cosine_sim(utility_matrix[3785].copy(), utility_matrix[17029].copy(), utility_matrix["mean_user_rating"].copy())}')

Adjusted Cosine Similarity between songID 3785 and songID 17029 is: 0.10000000000000016


## `iii` Pearson Correlation Coefficient
Finish implmenting the function below to calculate `Pearson Correlation Coefficient` between two vectors

In [ ]:
def pearson_sim(vec_a, vec_b):
    """
    Returns the pearson similarity score between two vectors.

            Parameters:
                vec_a (pandas.Series): Vector A
                vec_b (pandas.Series): Vector B

            Returns:
                sim_score (float): Similarity score between vectors vec_a and vec_b
    """
    vec_a=vec_a[:-1].copy()
    vec_b=vec_b[:-1].copy()
    common_songs = (vec_a != 0.0) & (vec_b != 0.0)
    vec_a_common_songs =vec_a[common_songs].copy()
    vec_b_common_songs =vec_b[common_songs].copy()
    vec_a_mean = vec_a_common_songs.mean()
    vec_b_mean = vec_b_common_songs.mean()
    adjusted_rating_a = vec_a_common_songs - vec_a_mean
    adjusted_rating_b = vec_b_common_songs - vec_b_mean
    dot_product=adjusted_rating_a @ adjusted_rating_b
    a=np.sqrt((np.square(vec_a_common_songs - vec_a_mean)).sum())
    b=np.sqrt((np.square(vec_b_common_songs - vec_b_mean)).sum())
    if a==0 or b==0:
        return 0
    sim_score = dot_product/(a*b)
    return sim_score

In [ ]:
print(f'Pearson Similarity between userID 56 and userID 227 is: {pearson_sim(utility_matrix.iloc[56].copy(), utility_matrix.iloc[227].copy())}')

Pearson Similarity between userID 56 and userID 227 is: 0


## `iv` Mean Squared Difference
Finish implmenting the function below to calculate `Mean Squared Difference` between two vectors

**Note**: Make sure you calculate the difference for common dimensions only (i.e. the dimensions both items/users have non-zero values in)

In [ ]:
def msd_sim(vec_a, vec_b):
    """
    Returns the mean squared difference similarity score between two vectors.
    Note: Only consider common items between the two vectors

            Parameters:
                vec_a (pandas.Series): Vector A
                vec_b (pandas.Series): Vector B

            Returns:
                sim_score (float): Similarity score between vectors vec_a and vec_b
    """
    common = (vec_a != 0.0) & (vec_b != 0.0)
    vec_a_common =vec_a[common].copy()
    vec_b_common =vec_b[common].copy()
    msd=np.mean(np.square(vec_a_common - vec_b_common))
    sim_score = 1/(msd+1)
    return sim_score

In [ ]:
print(f'MSD Similarity between userID 56 and userID 227 is: {msd_sim(utility_matrix.iloc[56,0:-1].copy(), utility_matrix.iloc[227,0:-1].copy())}')
print(f'MSD Similarity between songID 3785 and songID 17029 is: {msd_sim(utility_matrix[3785].copy(), utility_matrix[17029].copy())}')

MSD Similarity between userID 56 and userID 227 is: 1.0
MSD Similarity between songID 3785 and songID 17029 is: 0.6363636363636364


--------------------------

# `04` Collaborative Filtering

Practice for item-based collaborative filtering

## `0` Utility Matrix
Construct utility matrix for the loaded data `ratings`
- Songs as Index
- Users as Columns

In [ ]:
utility_matrix_2 = utility_matrix.T


In [ ]:
utility_matrix_2.head()

userID,0,4,5,7,14,20,31,33,40,46,...,199956,199969,199973,199974,199975,199976,199980,199988,199990,199996
songID,,,,,,,,,,,,,,,,,,,,,
2263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
3785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8063,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Item-Item Similarity Matrix

Construct item-item (Cosine/Adjusted Cosine) similarity matrix from the utility matrix  above.

In [ ]:
items = utility_matrix_2.iloc[0:-1].index
num_items = len(items)

In [ ]:
sim_df = pd.DataFrame(np.zeros((num_items, num_items)), index=items, columns=items)
for item1, item2 in combinations(items, 2):
    sim = cosine_sim(utility_matrix_2.loc[item1].copy(), utility_matrix_2.loc[item2].copy())
    sim_mat.at[item1, item2] = sim
    sim_mat.loc[item2, item1] = sim
np.fill_diagonal(sim_mat.values, 1)
sim_df

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
songID,,,,,,,,,,,,,,,,,,,,,
2263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
items = utility_matrix_2.iloc[0:-1].index
num_items = len(items)
sim_df = pd.DataFrame(np.zeros((num_items, num_items)), index=items, columns=items)
for item1, item2 in combinations(items, 2):
    sim = adjusted_cosine_sim(utility_matrix_2.loc[item1].copy(), utility_matrix_2.loc[item2].copy(),utility_matrix_2.iloc[-1])
    sim_mat.at[item1, item2] = sim
    sim_mat.loc[item2, item1] = sim
np.fill_diagonal(sim_mat.values, 1)
sim_mat

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
songID,,,,,,,,,,,,,,,,,,,,,
2263,1.000000,-0.840171,-0.985352,-0.751829,-0.795118,-0.831800,-0.985000,-0.732376,0.297233,-0.942990,...,-0.621742,-0.524369,0.261851,-0.753238,-0.897348,-0.600000,-0.720666,-0.689003,-0.746964,-0.402980
2726,-0.840171,1.000000,-0.739990,-0.556649,-0.687298,-0.346490,0.471940,-0.490940,-0.833521,-0.731370,...,-0.509797,-0.561312,-0.565003,-0.973084,-0.581170,-0.561936,-0.775611,-0.582638,-0.957590,-0.759481
3785,-0.985352,-0.739990,1.000000,-0.146223,-0.581939,-0.766446,-0.879459,0.100000,-0.675014,0.083787,...,0.000000,-0.373676,-0.767716,-0.674253,-0.953102,-1.000000,-0.924124,-0.638711,-0.264803,-0.812671
8063,-0.751829,-0.556649,-0.146223,1.000000,-0.724970,-0.512393,-0.702993,-0.743701,-0.313498,-0.856948,...,-0.199708,-0.671357,-0.669812,-0.273397,-0.111689,-0.624940,-0.298377,-0.495784,-0.845488,-0.187840
12709,-0.795118,-0.687298,-0.581939,-0.724970,1.000000,-0.768826,-0.903600,0.013425,-0.429509,-0.717180,...,-0.927170,-0.955667,-0.538737,-0.264413,-0.611919,-0.730715,-0.600000,-0.348722,-0.255909,-0.834742
13859,-0.831800,-0.346490,-0.766446,-0.512393,-0.768826,1.000000,-0.712928,-0.573854,-0.471831,-0.705024,...,-0.865057,-0.916326,-0.852108,-0.976195,-0.654701,-0.565095,-0.747692,-0.609364,-0.104042,-0.151050
16548,-0.985000,0.471940,-0.879459,-0.702993,-0.903600,-0.712928,1.000000,-0.544735,-0.652797,-0.495300,...,-0.669425,-0.761825,-0.908577,-0.947331,-0.678990,-0.782907,-0.862069,-0.227937,-0.529624,-0.577205
17029,-0.732376,-0.490940,0.100000,-0.743701,0.013425,-0.573854,-0.544735,1.000000,-0.943358,-0.985352,...,-0.700407,-0.705660,-0.703725,-0.753066,-0.263815,-0.473995,-0.417808,-0.407327,-0.531651,-0.645157
19299,0.297233,-0.833521,-0.675014,-0.313498,-0.429509,-0.471831,-0.652797,-0.943358,1.000000,-0.342110,...,-0.851823,-0.663696,-0.929917,-0.725160,-0.425393,-0.735948,-0.413860,-0.853153,-0.952328,-0.249203


## `ii` Candidate Generation and Filtering

Filter out items (user 199988) has rated from the similarity matrix above.

In [ ]:
#user_id = 199988
#potential_items = None
#potential_items

# Get the previously interacted items for user 199988 (excluding the last one)
prev_items=utility_matrix_2[utility_matrix_2[199988]!=0][199988].iloc[0:-1].index


In [ ]:
# Get the similarity scores for those previously interacted items
potential_items=sim_mat.loc[prev_items]
potential_items

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
songID,,,,,,,,,,,,,,,,,,,,,
2726,-0.840171,1.000000,-0.739990,-0.556649,-0.687298,-0.346490,0.471940,-0.490940,-0.833521,-0.731370,...,-0.509797,-0.561312,-0.565003,-0.973084,-0.581170,-0.561936,-0.775611,-0.582638,-0.957590,-0.759481
19299,0.297233,-0.833521,-0.675014,-0.313498,-0.429509,-0.471831,-0.652797,-0.943358,1.000000,-0.342110,...,-0.851823,-0.663696,-0.929917,-0.725160,-0.425393,-0.735948,-0.413860,-0.853153,-0.952328,-0.249203
43267,-0.851276,-0.681495,-0.013313,-0.551113,-0.446470,-0.605140,-0.779014,-0.732335,-0.682555,-0.542715,...,-0.499196,-0.678352,-0.445011,-0.837922,-0.508959,-0.387539,-0.519698,-0.481011,1.000000,-0.396801
56660,-0.956438,-0.637214,-0.399277,-0.740802,-0.516396,-0.613509,-0.577336,-0.819302,-0.556324,-0.345455,...,-0.976794,-0.646222,-0.646931,-0.754825,-0.285705,-0.694388,-0.507673,-0.498995,-0.652238,-0.543198


In [ ]:
filtered_sim_df = potential_items.drop(prev_items,axis=1)

In [ ]:
filtered_sim_df

songID,2263,3785,8063,12709,13859,16548,17029,19670,22763,24427,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
songID,,,,,,,,,,,,,,,,,,,,,
2726,-0.840171,-0.739990,-0.556649,-0.687298,-0.346490,0.471940,-0.490940,-0.731370,-0.706311,-0.298795,...,-0.509797,-0.561312,-0.565003,-0.973084,-0.581170,-0.561936,-0.775611,-0.582638,-0.957590,-0.759481
19299,0.297233,-0.675014,-0.313498,-0.429509,-0.471831,-0.652797,-0.943358,-0.342110,-0.482358,-0.655411,...,-0.851823,-0.663696,-0.929917,-0.725160,-0.425393,-0.735948,-0.413860,-0.853153,-0.952328,-0.249203
43267,-0.851276,-0.013313,-0.551113,-0.446470,-0.605140,-0.779014,-0.732335,-0.542715,-0.732060,0.241911,...,-0.499196,-0.678352,-0.445011,-0.837922,-0.508959,-0.387539,-0.519698,-0.481011,1.000000,-0.396801
56660,-0.956438,-0.399277,-0.740802,-0.516396,-0.613509,-0.577336,-0.819302,-0.345455,-0.713125,0.089579,...,-0.976794,-0.646222,-0.646931,-0.754825,-0.285705,-0.694388,-0.507673,-0.498995,-0.652238,-0.543198


## `iii` Top-K Candidate Selection

Selet top-K (a k of your choice) similar items for each item (user 199988) rated from the filtered similarity matrix above.

In [ ]:
top_n = filtered_sim_df.apply(lambda row: list(row.nlargest(3).index), axis=1)
print(top_n)

songID
2726      [16548, 42781, 62954]
19299      [52611, 94535, 2263]
43267    [132189, 79622, 24427]
56660     [36561, 74640, 24427]
dtype: object


In [ ]:
top_n_d = filtered_sim_df.apply(lambda row: row.nlargest(3).to_dict(), axis=1).to_dict()
print(top_n_d)
print(top_n_d[2726][16548])

{2726: {16548: 0.4719399037242694, 42781: 0.11527062757849706, 62954: -0.12696319036597545}, 19299: {52611: 1.0, 94535: 0.7049180327868851, 2263: 0.2972334978810871}, 43267: {132189: 1.0, 79622: 0.3927213716609408, 24427: 0.24191104219723272}, 56660: {36561: 0.1916393695788832, 74640: 0.12403473458920847, 24427: 0.08957854148304345}}
0.4719399037242694


In [ ]:
top_n_cleaned = list((filtered_sim_df.apply(pd.Series.nlargest, n=3, axis=1).columns))
print(top_n_cleaned)

[2263, 16548, 24427, 36561, 42781, 52611, 62954, 74640, 79622, 94535, 132189]


## `iv` Candidate Rating Prediction

Calculate the predicted rating for each of the candidate items.

In [ ]:
prev_items=utility_matrix_2[utility_matrix_2[199988]!=0][199988].iloc[0:-1].index
prev_items_rating=utility_matrix_2[utility_matrix_2[199988]!=0][199988].iloc[0:-1]
ratings=dict(zip(prev_items, prev_items_rating))
print(ratings)

{2726: 5.0, 19299: 5.0, 43267: 3.0, 56660: 5.0}


In [ ]:
predicted_ratings={}
for rec in top_n_cleaned:
    numerator=0
    denominator=0
    for prev in prev_items:
        if rec in top_n_d[prev]:
            numerator+= (top_n_d[prev][rec]*ratings[prev])
            denominator+= (top_n_d[prev][rec])
    if denominator!=0:
        predicted_ratings[rec]=round(numerator/denominator,2)

In [ ]:
predicted_ratings_df=pd.DataFrame(predicted_ratings.items(),columns=["recommended_song","predicted_rating"]).set_index("recommended_song")
predicted_ratings_df

,predicted_rating
recommended_song,
2263,5.00
16548,5.00
24427,3.54
36561,5.00
42781,5.00
52611,5.00
62954,5.00
74640,5.00
79622,3.00


------------------------------------------------------

# `05` Additional Tasks

## `i` Explore Surprise Library

- Install Scikit Surprise library.
- Explore the Library Documentation

In [ ]:
#pip install scikit-surprise


## `ii` Implement Item-Based KNN Approach

- Follow the steps explained in the sessions to prepare the KNN approach.
- Generate prediction ratings for user $199988$ on all songs.

In [ ]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)



In [ ]:
#Train-Test Split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
sim_options = {
    'name': 'cosine',
    'user_based': False
}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)


In [ ]:
#Generate Predictions for User 199988
# Get all item IDs (songs)
all_items = df['item_id'].unique()
# Filter items not yet rated by user 199988
rated_items = df[df['user_id'] == 199988]['item_id'].tolist()
unrated_items = [item for item in all_items if item not in rated_items]
# Predict ratings for all unrated items
predictions = []
for item_id in unrated_items:
    pred = model.predict(uid=199988, iid=item_id)
    predictions.append((item_id, pred.est))
# Convert to DataFrame for easy viewing
pred_df = pd.DataFrame(predictions, columns=['item_id', 'predicted_rating'])
pred_df = pred_df.sort_values(by='predicted_rating', ascending=False)
print(pred_df.head(10))  # Top 10 recommendations

----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Mahmoud \space Shawqi $$